In [ ]:
# gutendex.com/book

In [4]:
import requests
import pandas as pd

import os
from dotenv import load_dotenv

load_dotenv()

google_key = os.getenv("GOOGLE_BOOKS")
nyt_key = os.getenv("NYT")

In [5]:
# create a list of dataframes

dfs = []
url = 'https://gutendex.com/books'
for p in range(1,11):
    response = requests.get(url)
    data = response.json()
    dfs.append(pd.DataFrame.from_dict(data['results']))
    url = data['next']


In [6]:
df = pd.concat(dfs)
df.reset_index(inplace=True)
df.head(40)

,index,id,title,authors,summaries,translators,subjects,bookshelves,languages,copyright,media_type,formats,download_count
0,0,84,"Frankenstein; Or, The Modern Prometheus","[{'name': 'Shelley, Mary Wollstonecraft', 'bir...","[""Frankenstein; Or, The Modern Prometheus"" by ...",[],[Frankenstein's monster (Fictitious character)...,"[Browsing: Culture/Civilization/Society, Brows...",[en],False,Text,{'text/html': 'https://www.gutenberg.org/ebook...,117026
1,1,2701,"Moby Dick; Or, The Whale","[{'name': 'Melville, Herman', 'birth_year': 18...","[""Moby Dick; Or, The Whale"" by Herman Melville...",[],"[Adventure stories, Ahab, Captain (Fictitious ...","[Best Books Ever Listings, Browsing: Fiction, ...",[en],False,Text,{'text/html': 'https://www.gutenberg.org/ebook...,93457
2,2,1513,Romeo and Juliet,"[{'name': 'Shakespeare, William', 'birth_year'...","[""Romeo and Juliet"" by William Shakespeare is ...",[],"[Conflict of generations -- Drama, Juliet (Fic...","[Browsing: Fiction, Browsing: Literature, Brow...",[en],False,Text,{'text/html': 'https://www.gutenberg.org/ebook...,56729
3,3,1342,Pride and Prejudice,"[{'name': 'Austen, Jane', 'birth_year': 1775, ...","[""Pride and Prejudice"" by Jane Austen is a cla...",[],"[Courtship -- Fiction, Domestic fiction, Engla...","[Best Books Ever Listings, Browsing: Culture/C...",[en],False,Text,{'text/html': 'https://www.gutenberg.org/ebook...,54851
4,4,11,Alice's Adventures in Wonderland,"[{'name': 'Carroll, Lewis', 'birth_year': 1832...","[""Alice's Adventures in Wonderland"" by Lewis C...",[],[Alice (Fictitious character from Carroll) -- ...,"[Browsing: Children & Young Adult Reading, Bro...",[en],False,Text,{'text/html': 'https://www.gutenberg.org/ebook...,47514
5,5,100,The Complete Works of William Shakespeare,"[{'name': 'Shakespeare, William', 'birth_year'...","[""The Complete Works of William Shakespeare"" b...",[],[English drama -- Early modern and Elizabethan...,"[Browsing: Fiction, Browsing: Literature, Plays]",[en],False,Text,{'text/html': 'https://www.gutenberg.org/ebook...,42209
6,6,145,Middlemarch,"[{'name': 'Eliot, George', 'birth_year': 1819,...","[""Middlemarch"" by George Eliot is a novel writ...",[],"[Bildungsromans, City and town life -- Fiction...","[Best Books Ever Listings, Browsing: Culture/C...",[en],False,Text,{'text/html': 'https://www.gutenberg.org/ebook...,40160
7,7,2641,A Room with a View,"[{'name': 'Forster, E. M. (Edward Morgan)', 'b...","[""A Room with a View"" by E. M. Forster is a no...",[],"[British -- Italy -- Fiction, England -- Ficti...","[Browsing: Culture/Civilization/Society, Brows...",[en],False,Text,{'text/html': 'https://www.gutenberg.org/ebook...,39430
8,8,64317,The Great Gatsby,"[{'name': 'Fitzgerald, F. Scott (Francis Scott...","[""The Great Gatsby"" by F. Scott Fitzgerald is ...",[],"[First loves -- Fiction, Long Island (N.Y.) --...","[Browsing: Culture/Civilization/Society, Brows...",[en],False,Text,{'text/plain; charset=us-ascii': 'https://www....,39149
9,9,2542,A Doll's House : a play,"[{'name': 'Ibsen, Henrik', 'birth_year': 1828,...","[""A Doll's House"" by Henrik Ibsen is a three-a...",[],"[Man-woman relationships -- Drama, Marriage --...","[Best Books Ever Listings, Browsing: Fiction, ...",[en],False,Text,{'text/html': 'https://www.gutenberg.org/ebook...,39082


No publication date. Can I use the title to try and look up publication year with another tool?

In [7]:
df.summaries.iloc[0]

['"Frankenstein; Or, The Modern Prometheus" by Mary Wollstonecraft Shelley is a novel written in the early 19th century. The story explores themes of ambition, the quest for knowledge, and the consequences of man\'s hubris through the experiences of Victor Frankenstein and the monstrous creation of his own making.   The opening of the book introduces Robert Walton, an ambitious explorer on a quest to discover new lands and knowledge in the icy regions of the Arctic. In his letters to his sister Margaret, he expresses both enthusiasm and the fear of isolation in his grand venture. As Walton\'s expedition progresses, he encounters a mysterious, emaciated stranger who has faced great suffering—furthering the intrigue of his narrative. This stranger ultimately reveals his tale of creation, loss, and the profound consequences of seeking knowledge that lies beyond human bounds. The narrative is set up in a manner that suggests a deep examination of the emotions and ethical dilemmas faced by 

In [8]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [9]:
fig = px.scatter(df, x=df.index, y ='download_count', hover_name='title')
fig.show()

In [10]:
fig = px.bar(df.iloc[0:10,:], x='title', y = 'download_count')
fig.show()

This is kind of interesting, but it doesn't show downloads over time, sales, or any other metadata on the books to analyze. Crosslookup might be possible by using titles. 

Maybe NYT best seller list is a better gauge of what type of media our culture is consuming. This might be interesting to look at over time. I could compare this to say, popular podcasts, tv shows, movies, etc to see if our cultural interests have changed by media type.

In [11]:
ny_key = nyt_key
response = requests.get(f'https://api.nytimes.com/svc/books/v3/lists/overview.json?api-key={ny_key}')

data = response.json()

data['results'].keys()

dict_keys(['previous_published_date', 'published_date', 'next_published_date', 'published_date_description', 'bestsellers_date', 'lists', 'monthly_uri', 'weekly_uri'])

In [12]:
lists = {}
for list in data['results']['lists']:
    lists[(list['list_name'])] = list

lists.keys()

dict_keys(['Combined Print & E-Book Fiction', 'Combined Print & E-Book Nonfiction', 'Hardcover Fiction', 'Hardcover Nonfiction', 'Paperback Trade Fiction', 'Paperback Nonfiction', 'Advice, How-To and Miscellaneous', "Children's Middle Grade Hardcover", "Children's Picture Books", "Children's & Young Adult Series", 'Young Adult Hardcover', 'Audio Fiction', 'Audio Nonfiction', 'Business', 'Graphic Books and Manga', 'Mass Market', 'Middle Grade Paperback', 'Young Adult Paperback'])

In [13]:
lists['Combined Print & E-Book Fiction']['books'][0]['isbns']

[{'isbn10': '', 'isbn13': '9781668089330'}]

In [14]:
# date should be supplied as 2025-05-04

def get_titles(list,date=None):
    print(f"{list}\n-")

    if (date == None):
        response = requests.get(f'https://api.nytimes.com/svc/books/v3/lists/overview.json?api-key={ny_key}')

    else: 
        response = requests.get(f'https://api.nytimes.com/svc/books/v3/lists/overview.json?api-key={ny_key}&date={date}')



    for title in lists[list]['books']:
        print(title['title'])
        print(title['author'])
        print(title['description'])
        isbn= title['primary_isbn13']
        print(isbn)

        response = requests.get(f'https://books.googleapis.com/books/v1/volumes?q=isbn%3A{isbn}&key={google_key}')
        data = response.json()

        if (data['totalItems'] > 0):
            print(data['items'][0]['volumeInfo']['categories'])

        print('-')
    print("---")


get_titles('Hardcover Nonfiction')
get_titles('Hardcover Fiction')


Hardcover Nonfiction
-
ORIGINAL SIN
Jake Tapper and Alex Thompson
An account of Joe Biden’s initial decision to run for re-election in 2024 and its numerous consequences.
9798217060672
['Political Science']
-
THE ANXIOUS GENERATION
Jonathan Haidt
A co-author of “The Coddling of the American Mind” looks at the mental health impacts that a phone-based life has on children.
9780593655030
['Psychology']
-
MARK TWAIN
Ron Chernow
The Pulitzer Prize-winning biographer portrays the life and career of the literary celebrity and political pundit.
9780525561729
['Biography & Autobiography']
-
ABUNDANCE
Ezra Klein and Derek Thompson
A New York Times opinion columnist and a staff writer at The Atlantic evaluate obstacles to American progress.
9781668023488
['Business & Economics']
-
BIG DUMB EYES
Nate Bargatze
The Grammy Award-nominated comedian shares snippets from his life and career.
9781538768464
['Biography & Autobiography']
-
UNCOMMON FAVOR
Dawn Staley
The three-time Olympic gold medalist and

Some fiction books isbn don't match the nyt supplied one. Fiction books also don't have genre. Maybe Hardcover instead? Need to check when they can supply after getting API token.

https://docs.hardcover.app/api/guides/gettingbookdetails/

In [15]:
# Google books api key google_key
# example call GET https://www.googleapis.com/books/v1/volumes?q=flowers+inauthor:keyes&key=yourAPIKey

# This doesn't work and I don't know why
# response = requests.get(f'https://www.googleapis.com/books/v1/volumes?q=isbn:3A9781526635198&key={google_key}')

# This is what Books API query builder gave me.

response = requests.get(f'https://books.googleapis.com/books/v1/volumes?q=isbn%3A9798217060672&key={google_key}')


data = response.json()

data['items'][0]['volumeInfo']['categories']

['Political Science']